# Example of match data analysis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sf6_match_collection import scrapp_sf6_matches

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
PLAYER_SID = 1572500566 # MDZ_Jimmy
# PLAYER_SID: int = 3570388222 # Broski

scrapp_sf6_matches(PLAYER_SID)

This script will save on 'player_1572500566_sf6_matches.xlsx' file. Make sure it is closed and press ENTER to proceed.

Fetching the last 100 matches...


 50%|█████     | 5/10 [00:04<00:04,  1.13it/s]

In [ ]:
file_name = f"player_{PLAYER_SID}_sf6_matches.xlsx"

In [ ]:
data = pd.read_excel(file_name)
data["left_side"] = data["left_side"].map({0: "Right side", 1: "Left Side"})

last_day = str(data["uploaded_at"].max().date())

df_last_session = data[data["uploaded_at"].map(lambda d: str(d.date()) == last_day)]
data.head(n=5)

In [ ]:
def match_reports(df):
    
    sep = "\n" + 20 * "-" + "\n"
    
    print("Number of matches:", len(df), end=sep)
    
    pie_chart_format_high = lambda x: f"{round(x / 100 * len(df))}"
    pie_chart_format_low = lambda x: f"{round(x / 100 * len(df))} ({round(x, 2)} %)"
    
    
    
    for col in ["match_won", "replay_battle_type_name", "opposite_player_character", "left_side"]:
        pie_chart_format = pie_chart_format_high if col == "opposite_player_character" else pie_chart_format_low
        plt.figure()
        df[col].value_counts().plot.pie(
            autopct=pie_chart_format,
            figsize=(8, 6)
        )
        plt.title(col)
        plt.ylabel(None)
        
    print("% of match won", round(df["match_won"].mean() * 100, 2), "%", end=sep)
    print(df[["left_side", "match_won"]].groupby("left_side").mean(), end=sep)
    print(df[["opposite_player_character", "match_won"]].groupby("opposite_player_character").mean(), end=sep)
    
    print("Highest MR", df['main_player_mr'].max(), end=sep)
    print("Highest World Ranking", df['main_player_mr_ranking'].min(), end=sep)

    highest_winstreak = max(len(seq) for seq in "".join(df["match_won"].map(str)).split("0"))

    print("Highest winstreak (assuming no gap in match saved):", highest_winstreak, end=sep)
    
    df_ranked = df[df["replay_battle_type_name"] == "Ranked Match"]

    plt.figure(figsize=(8, 6))

    plt.plot(df_ranked["main_player_mr"], label="Main Player MR")
    plt.plot(df_ranked["opposite_player_mr"], label="Opposite Player MR")

    plt.grid(True)
    plt.legend()
    plt.xlabel("Match")
    plt.ylabel("MR")
    plt.title("Ranked matches MR Evolution")

## Latest session stats

In [ ]:
match_reports(df_last_session)

## Total matches stats

In [ ]:
match_reports(data)